In [2]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/SingleImageDataAug/BAGAN/BAGAN code"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
bagan_train_new.py	       data_augmentation.ipynb	rw
bagan_train.py		       Figures			selected_index_40.npz
balancing_gan.py	       LICENSE			utils.py
batch_generator_from_input.py  README.md
batch_generator.py	       run.sh


In [3]:
! pip install cifar10_web

In [4]:
!cp "/content/drive/My Drive/SingleImageDataAug/BAGAN/BAGAN code/." . -a

In [5]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [6]:
from bagan_train_new import *

Using TensorFlow backend.


# Generating

# Read

In [7]:
from cifar10_web import cifar10
import numpy as np

def get_indexes(index_list, label: int = 5, drop_ratio: float = 0.4):

    drop_ratio_list = dict(zip([0.4, 0.6, 0.75, 0.9],range(4)))
    
    return index_list[label+10*drop_ratio_list[drop_ratio]]

def get_cifar_10(return_one_hot_y: bool = False):
    
    X_train, y_train, X_test, y_test = cifar10(path=None)
    
    X_train = X_train.reshape(-1,3,32,32).transpose(0,2,3,1)
    X_test = X_test.reshape(-1,3,32,32).transpose(0,2,3,1)
        
    if return_one_hot_y == False:
        y_train = np.array([np.argmax(a, axis=0) for a in y_train])
        y_test = np.array([np.argmax(a, axis=0) for a in y_test])
        
    return X_train, y_train, X_test, y_test

def get_imbalanced_dataset(X_train, y_train, label, drop_ratio):
    
    if isinstance(label,int) and isinstance(drop_ratio, float):
        label = [label]
        drop_ratio = [drop_ratio]
    else:
        label = list(label)
        drop_ratio = list(drop_ratio)
        
    assert(len(label) == len(drop_ratio))
    assert(sum([1 if i in [0.4, 0.6, 0.75, 0.9] else 0 for i in drop_ratio]) == len(drop_ratio))
    
    npzfile = np.load("selected_index_40.npz", allow_pickle = True)
    indexes = npzfile["arr_0"]
    
    if y_train.ndim == 2:
        y_train_ = np.array([np.argmax(a, axis=0) for a in y_train])
    else:
        y_train_ = y_train

    for label_, drop_ratio_, i in zip(label, drop_ratio, range(len(label))):
        
        if i == 0:
            label_index = np.where(y_train_ == label_)[0]
            sample_index = get_indexes(indexes, label = label_, drop_ratio = drop_ratio_)
            deleted_index = np.delete(label_index, sample_index)
        else:
            label_index = np.where(y_train_ == label_)[0]
            sample_index = get_indexes(indexes, label = label_, drop_ratio = drop_ratio_)
            print(deleted_index.shape)
            print(np.delete(label_index, sample_index).shape)
            deleted_index = np.concatenate((deleted_index, np.delete(label_index, sample_index)))
    
        
    X_imbalanced = np.delete(X_train, deleted_index, 0)
    y_imbalanced = np.delete(y_train, deleted_index, 0)

    X_deleted = X_train[deleted_index]
    y_deleted = y_train[deleted_index]

    return X_imbalanced, y_imbalanced, X_deleted, y_deleted

In [8]:
X_train, y_train, X_test, y_test = get_cifar_10()
unbalance = [0.4]
target_classes = [2]
X_train_imbalanced ,y_train_imbalanced, X_deleted, y_deleted= get_imbalanced_dataset(X_train, y_train, label = target_classes, drop_ratio= unbalance)

Downloaded cifar-10-binary.tar.gz to /root/data/cifar10


In [ ]:
train_model(X_train_imbalanced ,y_train_imbalanced, X_test, y_test, unbalance = unbalance, target_classes = target_classes, output_dir = '/content/drive/My Drive/SingleImageDataAug/BAGAN', epochs=100, dataset_name='CIFAR10')

Executing BAGAN.
Using dataset:  CIFAR10
read input data...
input data loaded...
Required GAN for class [2]
Class counters:  [5000, 5000, 3000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
uratio set to: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
dratio set to: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
gratio set to: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
BAGAN init_autoenc
BAGAN: training autoencoder
Autoencoder train epoch: 1/100

Autoencoder train epoch: 2/100
Autoencoder train epoch: 3/100
Autoencoder train epoch: 4/100
Autoencoder train epoch: 5/100
Autoencoder train epoch: 6/100
Autoencoder train epoch: 7/100
Autoencoder train epoch: 8/100
Autoencoder train epoch: 9/100
Autoencoder train epoch: 10/100
Autoencoder train epoch: 11/100
Autoencoder train epoch: 12/100
Autoencoder train epoch: 13/100
Autoencoder train epoch: 14/100
Autoencoder train epoch: 15/100
Autoencoder train epoch: 16/100
Autoencoder